In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

fut_players = spark.table("workspace.default.fut_players_data")

df_brazil = fut_players.filter(F.col("nationality") == "Brazil") #filtrando por brazil

position_map_logic = (
    F.when(F.col("position") == "GK", "Goleiro")
     .when(F.col("position").isin(["CB", "LB", "RB", "LWB", "RWB"]), "Defesa")
     .when(F.col("position").isin(["CM", "CDM", "CAM", "LM", "RM"]), "Meio")
     .when(F.col("position").isin(["ST", "CF", "LW", "RW", "LF", "RF"]), "Ataque")
     .otherwise("Outros"))

df_with_groups = df_brazil.withColumn("position_group", position_map_logic)

df_in_formation = df_with_groups.filter(F.col("position_group") != "Outros")

window_spec = Window.partitionBy ("position_group").orderBy(F.col("overall").desc()) #usamos window porque nesse caso groupby não seria tão eficaz dado que nos retornaria apenas o melhor e nós queremos uma quantidade especifica de jogadores por posição

df_ranked = df_in_formation.withColumn("rank", F.row_number().over(window_spec))

dream_team = df_ranked.filter(
    (F.col("position_group") == "Goleiro") & (F.col("rank") <= 1) |
    (F.col("position_group") == "Defesa") & (F.col("rank") <= 4) |
    (F.col("position_group") == "Meio") & (F.col("rank") <= 4) |
    (F.col("position_group") == "Ataque") & (F.col("rank") <= 2))

final_dream_team = dream_team.select(
    "nationality", 
    "position_group", 
    "player_name", 
    "overall"
)

display(final_dream_team.orderBy("position_group"))

print(f"Total de jogadores no Time dos Sonhos: {final_dream_team.count()}")